# Imports

In [6]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
pd.set_option('display.max_colwidth', None)

# Functions

In [7]:
def analisar_resultados(df):

    total = len(df)

    df['model_answer'] = df['model_answer'].where(df['model_answer'].isin(['A', 'B', 'C', 'D']), 'invalid')

    acertos = (df['ANSWER'] == df['model_answer']).sum()
    invalidos = (df['model_answer'] == 'invalid').sum()
    erros = total - acertos - invalidos
    erros_com_invalidos = erros + invalidos

    pct_acertos = round((acertos / total) * 100, 2) if total > 0 else 0
    pct_erros = round((erros_com_invalidos / total) * 100, 2) if total > 0 else 0
    pct_invalidos = round((invalidos / total) * 100, 2) if total > 0 else 0

    print("Resumo:")
    print(f"✅ Acertos: {pct_acertos}% (sobre total)")
    print(f"❌ Erros + Inválidos: {pct_erros}% (sobre total)")
    print(f"🚫 Inválidos: {pct_invalidos}% (parte dos erros)")

    return {
        "acertos": acertos,
        "erros": erros,
        "invalidos": invalidos,
        "pct_acertos": pct_acertos,
        "pct_erros": pct_erros,
        "pct_invalidos": pct_invalidos
    }


def extrair_model_answer(texto):
    if not isinstance(texto, str):
        return 'invalid'
    match = re.search(r'[ABCD]', texto)
    return match.group(0) if match else 'invalid'

def analisar_resultados_com_grafico(df, titulo_extra=""):
    total = len(df)

    acertos = (df['ANSWER'] == df['model_answer']).sum()
    invalidos = (df['model_answer'] == 'invalid').sum()
    erros = total - acertos - invalidos

    proporcoes = {
        'Acertos': acertos / total,
        'Erros': erros / total,
        'Inválidos': invalidos / total
    }

    # Impressão dos percentuais
    for k, v in proporcoes.items():
        print(f"{k}: {v:.2%}")

    # Cores: verde (acerto), vermelho (erro), cinza (inválido)
    cores = ['#4CAF50', '#F44336', '#9E9E9E']

    plt.figure(figsize=(6, 6))
    plt.pie(
        proporcoes.values(),
        labels=proporcoes.keys(),
        autopct='%1.1f%%',
        startangle=140,
        colors=cores
    )
    plt.title(f'Distribuição de Respostas {titulo_extra}')
    plt.axis('equal')
    plt.show()

    return proporcoes


def processar_resultados(dir_path, df_answer, results, extrair_model_answer_func, dataset_name, tipo_fine_tune):
    model_names = os.listdir(dir_path)

    for filename in model_names:
        df = pd.read_csv(dir_path + "/" + filename)

        df['output_model'] = df['output_model'].str.upper()

        # Extrai texto depois de qualquer um dos padrões
        df['last_answer'] = df['output_model'].str.extract(
            r'(?:ASSISTANT:|ANSWER:|RESPOSTA:)\s*(.*)', flags=re.IGNORECASE
        ).fillna('invalid')  # ou use seu extrator depois

        df['model_answer'] = df['last_answer'].apply(extrair_model_answer_func)

        df_merged = pd.merge(df, df_answer, on='ID', how='right')

        print(f"\nShape do modelo {filename}: {df_merged.shape}")
        res = analisar_resultados(df_merged)

        results.append({
            'modelos': filename,
            'dataset': dataset_name,
            'tipo': tipo_fine_tune,
            **res
        })



def processar_resultados_english(dir_path, df_answer, results, extrair_model_answer_func, dataset_name, tipo_fine_tune):
    """
    Processa todos os modelos em um diretório e acumula resultados em uma lista externa.

    Parâmetros:
    - dir_path (str): Caminho dos arquivos CSV.
    - df_answer (DataFrame): Respostas corretas com colunas 'ID' e 'ANSWER'.
    - results (list): Lista acumuladora de resultados.
    - dataset (str): Nome do dataset (default: 'pt').
    - tipo (str): Tipo do modelo (default: 'sem_fine_tuning').

    Retorna:
    - None (resultados são adicionados à lista passada)
    """
    model_names = os.listdir(dir_path)

    for filename in model_names:
          
        df = pd.read_csv(dir_path +"/"+ filename )
     
        df['output_model'] = df['output_model'].str.upper()
        df['last_answer'] = df['output_model'].str.split('ANSWER:').str[-1]
        df['model_answer'] = df['last_answer'].apply(extrair_model_answer)

        df_merged = pd.merge(df, df_answer, on='ID', how='right')

        print(f"\nShape do modelo {filename}: {df_merged.shape}")
        res = analisar_resultados(df_merged)

        results.append({
            'modelos': filename,
            'dataset': dataset_name,
            'tipo': tipo_fine_tune,
            **res
        })

# Data Answers

In [8]:
df_answer = pd.read_csv('/home/annap/Documents/chatbot_copy/DATASETS/MMLU/0_mmlu_prompt_glosa_pt.csv')
df_answer = df_answer[['ID', 'ANSWER']]
answer_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
df_answer['ANSWER'] = df_answer['ANSWER'].map(answer_mapping)
print(df_answer.shape)
df_answer.head()

(15050, 2)


,ID,ANSWER
0,0,B
1,1,C
2,2,D
3,3,B
4,4,B


In [9]:
df = pd.read_csv("/home/annap/Documents/chatbot_copy/INFERENCE/DATASETS/UNSLOTH/V1/GLOSA/Qwen2.5-7B-Instruct-bnb-4bit-V1.csv")
df.head()

,ID,prompt,output_model
0,0,"Você é um assistente altamente capacitado. Leia a pergunta com atenção e escolha a resposta correta entre as opções (A), (B), (C) ou (D).\n\nPergunta:\nGRAU ENCONTRAR EXTENSÃO CAMPO Q RAIZ QUADRADA 2 RAIZ QUADRADA 3 RAIZ QUADRADA 18 SOBRE Q.\n\nOpções:\n(A) ZERO\n(B) 4\n(C) DOIS.\n(D) SEIS\n\nResposta:","você é um assistente altamente capacitado. leia a pergunta com atenção e escolha a resposta correta entre as opções (a), (b), (c) ou (d).\n\npergunta:\ngrau encontrar extensão campo q raiz quadrada 2 raiz quadrada 3 raiz quadrada 18 sobre q.\n\nopções:\n(a) zero\n(b) 4\n(c) dois.\n(d) seis\n\nresposta: b\nassistant: res"
1,1,"Você é um assistente altamente capacitado. Leia a pergunta com atenção e escolha a resposta correta entre as opções (A), (B), (C) ou (D).\n\nPergunta:\nP 1 2 5 4 2 3 S5 ÍNDICE ENCONTRAR P S5.\n\nOpções:\n(A) OITO\n(B) 2\n(C) VINTE QUATRO.\n(D) CENTO E VINTE\n\nResposta:","você é um assistente altamente capacitado. leia a pergunta com atenção e escolha a resposta correta entre as opções (a), (b), (c) ou (d).\n\npergunta:\np 1 2 5 4 2 3 s5 índice encontrar p s5.\n\nopções:\n(a) oito\n(b) 2\n(c) vinte quatro.\n(d) cento e vinte\n\nresposta: b\nassistant: res"
2,2,"Você é um assistente altamente capacitado. Leia a pergunta com atenção e escolha a resposta correta entre as opções (A), (B), (C) ou (D).\n\nPergunta:\nZERO TODOS ENCONTRAR CAMPO FINITO INDICADO POLINÔMIO COEFICIENTE X 5 MAIS 3 X 3 MAIS X 2 MAIS 2 X Z5.\n\nOpções:\n(A) ZERO\n(B) 1\n(C) ZERO VÍRGULA UM.\n(D) ZERO VÍRGULA QUATRO\n\nResposta:","você é um assistente altamente capacitado. leia a pergunta com atenção e escolha a resposta correta entre as opções (a), (b), (c) ou (d).\n\npergunta:\nzero todos encontrar campo finito indicado polinômio coeficiente x 5 mais 3 x 3 mais x 2 mais 2 x z5.\n\nopções:\n(a) zero\n(b) 1\n(c) zero vírgula um.\n(d) zero vírgula quatro\n\nresposta: a\nassistant: res"
3,3,"Você é um assistente altamente capacitado. Leia a pergunta com atenção e escolha a resposta correta entre as opções (A), (B), (C) ou (D).\n\nPergunta:\nAFIRMAÇÃO 1 GRUPO FATORES GRUPO NÃO-ABELIANO NÃO-ABELIANO. AFIRMAÇÃO 2 K SUBGRUPO NORMAL H H SUBGRUPO NORMAL G K SUBGRUPO NORMAL G.\n\nOpções:\n(A) VERDADE FATO.\n(B) MENTIROSO MENTIROSO\n(C) VERDADEIRO, FALSO.\n(D) FALSO VERDADEIRO\n\nResposta:","você é um assistente altamente capacitado. leia a pergunta com atenção e escolha a resposta correta entre as opções (a), (b), (c) ou (d).\n\npergunta:\nafirmação 1 grupo fatores grupo não-abeliano não-abeliano. afirmação 2 k subgrupo normal h h subgrupo normal g k subgrupo normal g.\n\nopções:\n(a) verdade fato.\n(b) mentiroso mentiroso\n(c) verdadeiro, falso.\n(d) falso verdadeiro\n\nresposta: a\nassistant: res"
4,4,"Você é um assistente altamente capacitado. Leia a pergunta com atenção e escolha a resposta correta entre as opções (A), (B), (C) ou (D).\n\nPergunta:\nPRODUTO ENCONTRAR POLINÔMIO ANEL POLINÔMIO F X 4 X MENOS 5 G X 2 X 2 MENOS 4 X MAIS 2 Z8 X.\n\nOpções:\n(A) DOIS X QUADRADO MAIS CINCO\n(B) 6X^2 4X 6 +\n(C) ZERO.\n(D) X DOIS MAIS UM\n\nResposta:","você é um assistente altamente capacitado. leia a pergunta com atenção e escolha a resposta correta entre as opções (a), (b), (c) ou (d).\n\npergunta:\nproduto encontrar polinômio anel polinômio f x 4 x menos 5 g x 2 x 2 menos 4 x mais 2 z8 x.\n\nopções:\n(a) dois x quadrado mais cinco\n(b) 6x^2 4x 6 +\n(c) zero.\n(d) x dois mais um\n\nresposta: d\nassistant: res"


In [10]:
results = []

# Portuguese

É o dataset em português, onde foi passado nos modelos do unsloth.

In [11]:
dir_path_pt = '../UNSLOTH/NO_FINE_TUNE/PT'
dir_path_pt_v1 = '../UNSLOTH/V1/PT_PT'
dir_path_pt_v2 = '../UNSLOTH/V2/PT_PT'


print("Sem Fine Tuning")
processar_resultados(
    dir_path_pt,
    df_answer,
    results,
    extrair_model_answer,
    'pt',
    'sem_fine_tuning'  
)

print("Fine Tuning V1")
processar_resultados(
    dir_path_pt_v1,
    df_answer,
    results,
    extrair_model_answer,
    'pt',
    'v1'  
)

print("Fine Tuning V2")
processar_resultados(
    dir_path_pt_v2,
    df_answer,
    results,
    extrair_model_answer,
    'pt',
    'v2'  
)

Sem Fine Tuning

Shape do modelo gemma-3-12b-it-unsloth-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 60.93% (sobre total)
❌ Erros + Inválidos: 39.07% (sobre total)
🚫 Inválidos: 1.86% (parte dos erros)

Shape do modelo zephyr-sft-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 45.09% (sobre total)
❌ Erros + Inválidos: 54.91% (sobre total)
🚫 Inválidos: 0.07% (parte dos erros)

Shape do modelo Qwen3-14B-unsloth-bnb-4bit-unsloth-largest.csv: (15050, 6)
Resumo:
✅ Acertos: 26.71% (sobre total)
❌ Erros + Inválidos: 73.29% (sobre total)
🚫 Inválidos: 3.24% (parte dos erros)

Shape do modelo Llama-3.2-3B-Instruct-unsloth-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 44.19% (sobre total)
❌ Erros + Inválidos: 55.81% (sobre total)
🚫 Inválidos: 3.23% (parte dos erros)

Shape do modelo Llama-3.1-8B-unsloth-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 24.81% (sobre total)
❌ Erros + Inválidos: 75.19% (sobre total)
🚫 Inválidos: 46.64% (parte dos erros)

Shape do modelo Qwen2.5-7B-Instruct-bnb-4bit.csv: (15050, 

# Glosa

In [12]:
dir_path_gl = '../UNSLOTH/NO_FINE_TUNE/GLOSA'
dir_path_gl_v1 = '../UNSLOTH/V1/GLOSA'
dir_path_gl_v2 = '../UNSLOTH/V2/GLOSA'

print("Sem Fine Tuning")
processar_resultados(
    dir_path_gl,
    df_answer,
    results,
    extrair_model_answer,
    'glosa',
    'sem_fine_tuning'  
)

print("Fine Tuning V1")
processar_resultados(
    dir_path_gl_v1,
    df_answer,
    results,
    extrair_model_answer,
    'glosa',
    'v1'  
)

print("Fine Tuning V2")
processar_resultados(
    dir_path_gl_v2,
    df_answer,
    results,
    extrair_model_answer,
    'glosa',
    'v2'  
)

Sem Fine Tuning

Shape do modelo gemma-3-12b-it-unsloth-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 45.29% (sobre total)
❌ Erros + Inválidos: 54.71% (sobre total)
🚫 Inválidos: 0.47% (parte dos erros)

Shape do modelo zephyr-sft-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 33.82% (sobre total)
❌ Erros + Inválidos: 66.18% (sobre total)
🚫 Inválidos: 0.01% (parte dos erros)

Shape do modelo Qwen3-14B-unsloth-bnb-4bit-unsloth-largest.csv: (15050, 6)
Resumo:
✅ Acertos: 24.24% (sobre total)
❌ Erros + Inválidos: 75.76% (sobre total)
🚫 Inválidos: 0.9% (parte dos erros)

Shape do modelo Llama-3.2-3B-Instruct-unsloth-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 33.77% (sobre total)
❌ Erros + Inválidos: 66.23% (sobre total)
🚫 Inválidos: 0.88% (parte dos erros)

Shape do modelo Llama-3.1-8B-unsloth-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 14.58% (sobre total)
❌ Erros + Inválidos: 85.42% (sobre total)
🚫 Inválidos: 53.84% (parte dos erros)

Shape do modelo Qwen2.5-7B-Instruct-bnb-4bit.csv: (15050, 6

# English

Esse tem que fazer para o v1 e o V2 tbm 

In [13]:
dir_path_eng = '../UNSLOTH/NO_FINE_TUNE/ENG'
dir_path_eng_v1 = '../UNSLOTH/V1/ENG'
dir_path_eng_v2 = '../UNSLOTH/V2/ENG'


print("Sem Fine Tuning")
processar_resultados(
    dir_path_eng,
    df_answer,
    results,
    extrair_model_answer,
    'english',
    'sem_fine_tuning'  
)

print("Fine Tuning V1")
processar_resultados(
    dir_path_eng_v1,
    df_answer,
    results,
    extrair_model_answer,
    'english',
    'v1'  
)

print("Fine Tuning V2")
processar_resultados(
    dir_path_eng_v2,
    df_answer,
    results,
    extrair_model_answer,
    'english',
    'v2'  
)

Sem Fine Tuning

Shape do modelo gemma-3-12b-it-unsloth-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 71.34% (sobre total)
❌ Erros + Inválidos: 28.66% (sobre total)
🚫 Inválidos: 0.06% (parte dos erros)

Shape do modelo zephyr-sft-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 56.91% (sobre total)
❌ Erros + Inválidos: 43.09% (sobre total)
🚫 Inválidos: 0.16% (parte dos erros)

Shape do modelo Qwen3-14B-unsloth-bnb-4bit-unsloth-largest.csv: (15050, 6)
Resumo:
✅ Acertos: 28.91% (sobre total)
❌ Erros + Inválidos: 71.09% (sobre total)
🚫 Inválidos: 4.91% (parte dos erros)

Shape do modelo Llama-3.2-3B-Instruct-unsloth-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 59.34% (sobre total)
❌ Erros + Inválidos: 40.66% (sobre total)
🚫 Inválidos: 0.07% (parte dos erros)

Shape do modelo Llama-3.1-8B-unsloth-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 54.76% (sobre total)
❌ Erros + Inválidos: 45.24% (sobre total)
🚫 Inválidos: 0.5% (parte dos erros)

Shape do modelo Qwen2.5-7B-Instruct-bnb-4bit.csv: (15050, 6)

# GLOSA PT

In [14]:
dir_path_gl_pt = '../UNSLOTH/NO_FINE_TUNE/GLOSA_PT'
dir_path_gl_pt_v1 = '../UNSLOTH/V1/GLOSA_PT'
dir_path_gl_pt_v2 = '../UNSLOTH/V2/GLOSA_PT'

print("Sem Fine Tuning")
processar_resultados(
    dir_path_gl_pt,
    df_answer,
    results,
    extrair_model_answer,
    'glosa_pt',
    'sem_fine_tuning'  
)

print("Fine Tuning V1")
processar_resultados(
    dir_path_gl_pt_v1,
    df_answer,
    results,
    extrair_model_answer,
    'glosa_pt',
    'v1'  
)

print("Fine Tuning V2")
processar_resultados(
    dir_path_gl_pt_v2,
    df_answer,
    results,
    extrair_model_answer,
    'glosa_pt',
    'v2'  
)

Sem Fine Tuning

Shape do modelo gemma-3-12b-it-unsloth-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 52.72% (sobre total)
❌ Erros + Inválidos: 47.28% (sobre total)
🚫 Inválidos: 0.46% (parte dos erros)

Shape do modelo zephyr-sft-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 38.4% (sobre total)
❌ Erros + Inválidos: 61.6% (sobre total)
🚫 Inválidos: 0.02% (parte dos erros)

Shape do modelo Qwen3-14B-unsloth-bnb-4bit-unsloth-largest.csv: (15050, 6)
Resumo:
✅ Acertos: 24.65% (sobre total)
❌ Erros + Inválidos: 75.35% (sobre total)
🚫 Inválidos: 0.86% (parte dos erros)

Shape do modelo Llama-3.2-3B-Instruct-unsloth-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 38.8% (sobre total)
❌ Erros + Inválidos: 61.2% (sobre total)
🚫 Inválidos: 0.84% (parte dos erros)

Shape do modelo Llama-3.1-8B-unsloth-bnb-4bit.csv: (15050, 6)
Resumo:
✅ Acertos: 15.76% (sobre total)
❌ Erros + Inválidos: 84.24% (sobre total)
🚫 Inválidos: 57.76% (parte dos erros)

Shape do modelo Qwen2.5-7B-Instruct-bnb-4bit.csv: (15050, 6)
R

# Results

In [15]:
df_results = pd.DataFrame(results)
df_results.tail(10)

,modelos,dataset,tipo,acertos,erros,invalidos,pct_acertos,pct_erros,pct_invalidos
82,gemma-3-12b-it-bnb-4bit-V1.csv,glosa_pt,v1,7934,7047,69,52.72,47.28,0.46
83,Llama-3.2-3B-Instruct-unsloth-bnb-4bit-V1.csv,glosa_pt,v1,5765,9162,123,38.31,61.69,0.82
84,Qwen2.5-14B-Instruct-unsloth-bnb-4bit-V1.csv,glosa_pt,v1,8215,6713,122,54.58,45.42,0.81
85,Qwen2.5-7B-Instruct-bnb-4bit-V2.csv,glosa_pt,v2,5613,7094,2343,37.30,62.70,15.57
86,Llama-3.2-3B-Instruct-unsloth-bnb-4bit-V2.csv,glosa_pt,v2,5446,9435,169,36.19,63.81,1.12
87,gemma-3-12b-it-bnb-4bit-V2.csv,glosa_pt,v2,7934,7047,69,52.72,47.28,0.46
88,zephyr-sft-bnb-4bit-V2.csv,glosa_pt,v2,4074,10840,136,27.07,72.93,0.90
89,phi-4-unsloth-bnb-4bit-V2.csv,glosa_pt,v2,6541,8186,323,43.46,56.54,2.15
90,Llama-3.1-8B-unsloth-bnb-4bit-V2.csv,glosa_pt,v2,5206,9695,149,34.59,65.41,0.99
91,Qwen2.5-14B-Instruct-unsloth-bnb-4bit-V2.csv,glosa_pt,v2,7283,7593,174,48.39,51.61,1.16


In [16]:
df_results.to_csv("results_all_models.csv", index=False)